for permissions in .sh files run "chmod u+x filename.sh"

##Terminal 1

docker-compose up -d  
./step1_spinup.sh

this will: 
-Create topic "events"
-Describe Topic "events"
-Spin Up Game game_api.py


##Terminal 2

./step2_consumer.sh

this will: 
-Launch Kafka Consumer for topic "events"


##Terminal 3

./step3_producemessages.sh

this will: 
-Run a watch command for benchevents.sh that will create a random event every 3 seconds while running.



In [1]:
import json
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
import time

In [9]:
extracted_join_guild_events = spark.read.parquet('/tmp/join_guild')
extracted_join_guild_events = extracted_join_guild_events.rdd.map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw_event))).toDF()
extracted_join_guild_events.registerTempTable("extracted_join_guild_events")
if spark._jsparkSession.catalog().tableExists("default", "join_guild"):
    query = """
            insert overwrite table join_guild
            select * from extracted_join_guild_events
        """
else:
    query = """
            create external table join_guild
            stored as parquet
            location '/tmp/hive_join_guild'
            as
            select * from extracted_join_guild_events
        """
spark.sql(query)


DataFrame[]